# 📊 Phân tích dữ liệu

Gồm các phần:
- Thống kê mô tả
- Phân phối dữ liệu
- Tương quan
- Phân tích xu hướng theo thời gian
- Phân tích City - Restaurant
- Phân tích Customer - Payment
- Đề xuất

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../data/foodpanda_orders.csv')
df.head()

## Thống kê mô tả mở rộng

In [ ]:
desc = df.describe(include='all').T
desc['median'] = df.median(numeric_only=True)
desc['skewness'] = df.skew(numeric_only=True)
desc['kurtosis'] = df.kurtosis(numeric_only=True)
desc

## Kiểm tra phân phối dữ liệu

In [ ]:
numeric_cols = df.select_dtypes(include=['int64','float64']).columns

for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True)
    plt.title(f'Phân phối của {col}')
    plt.show()

## Tương quan giữa các biến

In [ ]:
corr = df.corr(numeric_only=True)
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Heatmap tương quan')
plt.show()

## Phân tích xu hướng theo thời gian

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['month'] = df['order_date'].dt.to_period('M')

monthly = df.groupby('month')['total_amount'].sum()
monthly.plot(figsize=(8,4))
plt.title('Doanh thu theo tháng')
plt.ylabel('Doanh thu')
plt.show()

## Phân tích City - Restaurant

In [ ]:
city_sales = df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
city_sales.plot(kind='bar', figsize=(8,4))
plt.title('Doanh thu theo thành phố')
plt.show()

top_rest = df.groupby('restaurant_name')['total_amount'].sum().sort_values(ascending=False).head(10)
top_rest.plot(kind='bar', figsize=(8,4))
plt.title('Top 10 nhà hàng doanh thu cao nhất')
plt.show()

## Phân tích Customer - Payment

In [ ]:
pay_count = df['payment_method'].value_counts()
pay_count.plot(kind='pie', autopct='%1.1f%%', figsize=(6,6))
plt.title('Phương thức thanh toán')
plt.ylabel('')
plt.show()

customer_value = df.groupby('customer_id')['total_amount'].sum().sort_values(ascending=False).head(10)
customer_value.plot(kind='bar', figsize=(8,4))
plt.title('Top 10 khách hàng giá trị nhất')
plt.show()

## Kết luận – Đề xuất

### 1️. Phân phối lệch phải
- Các biến doanh thu và thời gian giao hàng bị lệch -> cân nhắc log-transform.

### 2️. City - Restaurant
- Tập trung nhiều tài xế ở thành phố top doanh thu.
- Hợp tác ưu tiên với nhóm restaurant top 10.

### 3️. Customer - Payment
- Khuyến khích thanh toán online nếu COD quá cao.
- Chạy chương trình chăm sóc khách hàng VIP (top 5%).

### 4. Trends theo thời gian
- Nếu doanh thu giảm ở vài tháng -> chạy chiến dịch khuyến mãi.
